Datadog API only allows filtering by hostname.
That's different than the AWS ID.

To filter by AWS ID, looking at downloading the full host data and making key-value pairs of
`hostname-aws_id` that can be reversed.

For example, I have a host that has AWS ID `i-069a78`.
Its hostname is also i-... by coincidence following the rules of
https://docs.datadoghq.com/agent/faq/how-datadog-agent-determines-the-hostname/?tab=agentv6v7

Here's a subset of the api result for this host

```
{
	'name': 'i-069a78',
	'tags_by_source': {
		'Datadog': ['host:i-069a78'],
		'Users': ['client:bsec', 'host:abcdef']
	},
	'meta': {
		'socket-hostname': 'ip-123-123-123-123',
		'host_id': 11111111,
		'socket-fqdn': 'ip-123-123-123-123.us-west-2.compute.internal.',
	},
	'host_name': 'i-069a78',
	'aws_id': 'i-069a78',
	'id': 11111111,
	'aliases': ['ip-123-123-123-123.us-west-2.compute.internal', 'i-069a78']
}
```

In [ ]:
# Python 3.6.8 (default, Aug 20 2019, 17:12:48)
import datadog
datadog.initialize()
datadog_api = datadog.api

In [ ]:
# download full hostname-awsid map and reverse it
h_all = datadog_api.Hosts.search()
assert len(h_all['host_list']) > 0
assert h_all['total_returned'] > 0

# could use 'name' instead of 'host_name' in the below
h_all = {x['aws_id']: x['host_name'] for x in h_all['host_list']}
h_all

In [ ]:
# go from aws_id to host_name
aws_id = 'i-069a78' # real id truncated
assert aws_id in h_all
host_id = h_all[aws_id]
host_id

In [ ]:
h_search_filter = datadog_api.Hosts.search(filter='host:%s'%host_id)
assert len(h_search_filter['host_list']) > 0
assert h_search_filter['total_returned'] == 1
# assert h_search_filter['host_list'][0]['name']==host_id
assert h_search_filter['host_list'][0]['host_name']==host_id

h_search_filter